# 6.9 Map Reduce LCEL Chain

이번에는 LCEL을 이용해서 Map Reduce chain을 직접 구현해볼 것이다.

일단은 단순화된 버전을 만들 것이다. 복잡한 기능의 추가는 나중에 얼마든지 가능하기 때문이다.<br>
token을 세어주거나 prompt가 context window에 적합한지 확인하는 그런거 말이다. 일단 단순하게 하나 만들어보면, 어떻게 동작하는지 이해할 수 있고, 직접 만들 수 있다는 확신도 갖게 된다.<br><br>

Map Reduce가 어떤식으로 작동하는지에대해 알아보자.<br>
일단은 질문과 관련된 document의 list를 얻어야 한다. 그 다음으로는 list내부의 모든 document들을 위한 prompt를 만들어 줄 것이다. 그 prompt는 LLM에게 전달할건데, 기본적인 내용은 다음과 같다.

    '이 document를 읽고, 사용자의 질문에 답변하기에 적절한 정보가 있는지 확인하고, 있다면 추출해 주세요.'

이를 전달받은 LLM은 응답(response)을 출력할 것이다. 그리고 LLM으로부터 받은 response들을 취합해 하나의 document를 만들어낼 것이다. 즉, 하나의 document에 합친다는 말이다.<br>
그렇게 만들어진 단 하나의 최종 document가, LLM을 위한 prompt로 전달될 것이다. 전달될때의 prompt내용은 다음과 같을 것이다.

    '이것은 질문과 관련이 있는 정보들 입니다. 이를 사용하여 대답해주세요'

이렇게 되면 마침내 처음의 질문에 대한 답변이 생성될 것이다.

<br>

그렇다면 현재 이 방식과 stuff중 어떤 상황에서 어느것을 사용하는 것이 더 효율적일까? 정답은 바로, 우리가 원하는 prompt의 크기와 검색할 document의 수에 따라 달라진다. 만약 retriever가 검색 결과로 천 개이상의 document를 반환한다면, stuff는 사용할 수 없다. 왜냐하면 stuff의 prompt에 그 document들을 모드 넣을 수 없기 때문이다. <br>
바로 이런 상황이 Map Reduce방식이 빛을 발하는 순간이다.

나중에 우리가 회의 GPT를 만들때에도 이런 형식의 코드로 구현할 것이다. 

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

llm = ChatOpenAI(temperature=0.1)

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("../files/chapter_one.docx")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cache_dir = LocalFileStore("./.cache")

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
     embeddings,
     cache_dir,
)

vectorstore = Chroma.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()


map_doc_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        """
        Use the following portion of a long document to see if any of the text is relevant to anser the question
        Return any relevant text verbatim.
        ------
        {context}
        """
    ),
    ("human", "{question}")
])

map_doc_chain = map_doc_prompt | llm

def map_docs(inputs):
    documents = inputs['documents']
    question = inputs['question']
    results = []
    for document in documents:
        result = map_doc_chain.invoke({
            "context": document.page_content,
            "question": question
        }).content
        results.append(result)
    results = "\n\n".join(results)
    return results

map_chain = {"documents": retriever, "question": RunnablePassthrough()} | RunnableLambda(map_docs)


final_prompt = ChatPromptTemplate.from_messages([
    (
        "system", 
        """
        Given the following extracted parts of a long docement and a question, create a final anwer.
        If you don't know the answer, just say that you don't know.
        Don't try to make up an answer.
        ------
        {context}
        """
    ),
    ("human", "{question}")
])

chain = {"context": map_chain, "question": RunnablePassthrough()} | final_prompt | llm

chain.invoke("What is the name of Oldwyn’s mechanical doll?")

AIMessage(content="The name of Oldwyn's mechanical doll is Vesper.")

## 1. 문서 내 관련 내용 추출을 위한 프롬프트 생성 (라인 37 ~ 48)

- **라인 37:**  
  ```python
  map_doc_prompt = ChatPromptTemplate.from_messages([
  ```  
  - **역할:**  
    - `ChatPromptTemplate.from_messages` 메서드를 사용하여, LLM(대형 언어 모델)에게 전달할 메시지 템플릿을 생성합니다.  
    - 이 템플릿은 이후 각 문서 조각의 내용(context)와 질문(question)을 LLM에 전달할 때 사용됩니다.

- **라인 38 ~ 46:**  
  ```python
      (
          "system",
          """
          Use the following portion of a long document to see if any of the text is relevant to anser the question
          Return any relevant text verbatim.
          ------
          {context}
          """
      ),
  ```  
  - **역할:**  
    - 첫 번째 메시지는 `"system"` 역할로, LLM에게 시스템 지시사항을 전달합니다.
    - 멀티라인 문자열 안에 다음의 내용이 포함되어 있습니다:  
      - **지시사항:**  
        - “Use the following portion of a long document to see if any of the text is relevant to anser the question”  
          → 주어진 문서 일부에서 질문에 관련된 내용을 찾으라는 명령입니다.
        - “Return any relevant text verbatim.”  
          → 관련 있는 텍스트가 있다면 원문 그대로 반환하라는 지시입니다.
        - “------” 구분선과 함께 `{context}` 플레이스홀더가 있어, 이후 실제 문서 내용이 이 자리에 삽입됩니다.

- **라인 47:**  
  ```python
      ("human", "{question}")
  ```  
  - **역할:**  
    - 두 번째 메시지는 `"human"` 역할로, 실제 사용자의 질문이 들어갈 부분을 `{question}` 플레이스홀더로 지정합니다.
    - 이 구조는 prompt에 “시스템 지시사항”과 “사용자 질문” 두 부분을 모두 포함하게 됩니다.

- **라인 48:**  
  ```python
  ])
  ```  
  - **역할:**  
    - 메시지 리스트를 닫아서, `map_doc_prompt` 템플릿 구성이 완료됨을 나타냅니다.

---

## 2. 문서 관련 추출 체인 구성 (라인 50)

- **라인 50:**  
  ```python
  map_doc_chain = map_doc_prompt | llm
  ```  
  - **역할:**  
    - 위에서 생성한 `map_doc_prompt` 템플릿과 LLM 인스턴스(`llm`)를 파이프(`|`) 연산자로 연결합니다.
    - **처리 과정:**  
      1. 입력받은 `context`와 `question`을 템플릿에 채워서 메시지를 구성합니다.
      2. 그 메시지를 LLM에 전달하여, 각 문서 조각에서 질문과 관련된 텍스트를 추출하도록 요청합니다.
    - **출력:**  
      - 각 호출 시, LLM이 반환한 응답 객체가 생성됩니다.

---

## 3. 여러 문서에 대한 매핑 함수 정의 (라인 52 ~ 63)

- **라인 52:**  
  ```python
  def map_docs(inputs):
  ```  
  - **역할:**  
    - `map_docs`라는 함수를 정의합니다.  
    - 이 함수는 이후 여러 문서를 받아서 각 문서에 대해 `map_doc_chain`을 호출하는 역할을 합니다.

- **라인 53:**  
  ```python
      documents = inputs['documents']
  ```  
  - **역할:**  
    - `inputs` 딕셔너리에서 `documents` 키에 해당하는 값을 추출합니다.
    - **의미:**  
      - 이 값은 여러 문서(또는 문서 조각)들의 리스트여야 합니다.

- **라인 54:**  
  ```python
      question = inputs['question']
  ```  
  - **역할:**  
    - `inputs` 딕셔너리에서 `question` 키에 해당하는 값을 추출합니다.
    - **의미:**  
      - 이 값은 사용자가 던진 질문(문자열)입니다.

- **라인 55:**  
  ```python
      results = []
  ```  
  - **역할:**  
    - 결과를 저장할 빈 리스트를 초기화합니다.
    - **의미:**  
      - 각 문서에 대해 LLM의 응답(관련 텍스트)을 모아둘 공간입니다.

- **라인 56:**  
  ```python
      for document in documents:
  ```  
  - **역할:**  
    - 문서 리스트에 있는 각 문서를 순회하면서 처리합니다.

- **라인 57 ~ 60:**  
  ```python
          result = map_doc_chain.invoke({
              "context": document.page_content,
              "question": question
          }).content
  ```  
  - **세부 설명:**  
    - **라인 57:**  
      - `map_doc_chain.invoke({...})`를 호출하여, 각 문서에 대해 LLM에게 질문과 문서의 내용을 전달합니다.
    - **라인 58:**  
      - `"context": document.page_content`  
        - 각 문서 객체의 `page_content` 속성을 사용하여, 실제 텍스트 내용을 `context` 자리에 넣습니다.
    - **라인 59:**  
      - `"question": question`  
        - 앞서 추출한 질문을 `question` 자리에 넣습니다.
    - **라인 60:**  
      - `.content`를 통해 LLM의 응답에서 실제 텍스트(문자열)를 추출합니다.
    - **의미:**  
      - 각 문서에 대해, 해당 문서 내용 중 질문과 관련된 부분을 LLM이 찾아 반환하도록 합니다.

- **라인 61:**  
  ```python
          results.append(result)
  ```  
  - **역할:**  
    - LLM으로부터 받은 결과(관련 텍스트)를 `results` 리스트에 추가합니다.

- **라인 62:**  
  ```python
      results = "\n\n".join(results)
  ```  
  - **역할:**  
    - 리스트에 저장된 각 결과들을 두 줄의 개행 문자("\n\n")로 합쳐 하나의 큰 문자열로 만듭니다.
    - **의미:**  
      - 여러 문서에서 추출된 결과를 하나의 문맥(context)으로 통합합니다.

- **라인 63:**  
  ```python
      return results
  ```  
  - **역할:**  
    - 통합된 결과 문자열을 반환합니다.
    - **의미:**  
      - 이 함수의 최종 출력은 모든 문서에서 추출된 관련 텍스트가 결합된 하나의 문자열입니다.

---

## 4. 매핑 체인 구성 (라인 65)

- **라인 65:**  
  ```python
  map_chain = {"documents": retriever, "question": RunnablePassthrough()} | RunnableLambda(map_docs)
  ```  
  - **역할:**  
    - 새로운 체인(`map_chain`)을 구성합니다.
  - **세부 설명:**  
    - **입력 구성:**  
      - 딕셔너리 `{ "documents": retriever, "question": RunnablePassthrough() }`  
        - `"documents": retriever`  
          - `retriever`는 벡터 스토어로부터 관련 문서들을 검색하는 역할을 합니다.
          - 이 체인은 최종 질문을 받아 관련 문서를 찾아서 반환합니다.
        - `"question": RunnablePassthrough()`  
          - `RunnablePassthrough`는 입력된 질문을 그대로 통과시킵니다.
    - **파이프 연산자(`|`)로 연결:**  
      - 이후 `RunnableLambda(map_docs)`와 연결되어, 위 딕셔너리의 데이터를 `map_docs` 함수에 전달합니다.
    - **최종 처리:**  
      - `map_chain`은 주어진 질문을 이용하여 retriever에서 문서를 검색하고, 각 문서에 대해 `map_docs` 함수를 실행하여 관련 텍스트들을 추출 및 통합합니다.
    - **출력:**  
      - 통합된 관련 텍스트(문자열)가 반환됩니다.

---

## 5. 최종 답변 생성을 위한 프롬프트 생성 (라인 67 ~ 79)

- **라인 67:**  
  ```python
  final_prompt = ChatPromptTemplate.from_messages([
  ```  
  - **역할:**  
    - 최종 답변을 생성하기 위한 또 다른 프롬프트 템플릿을 생성합니다.
    - 이 템플릿은 이전 단계에서 추출된 텍스트(문맥)와 질문을 사용해 LLM이 최종 답변을 생성하도록 안내합니다.

- **라인 68 ~ 77:**  
  ```python
      (
          "system", 
          """
          Given the following extracted parts of a long docement and a question, create a final anwer.
          If you don't know the answer, just say that you don't know.
          Don't try to make up an answer.
          ------
          {context}
          """
      ),
  ```  
  - **역할:**  
    - `"system"` 메시지로 LLM에게 최종 답변 생성 방법에 대한 지시사항을 제공합니다.
  - **세부 내용:**  
    - **지시사항:**  
      - “Given the following extracted parts of a long docement and a question, create a final anwer.”  
        → 추출된 문맥과 질문을 바탕으로 최종 답변을 만들어내라는 명령입니다.
      - “If you don't know the answer, just say that you don't know.”  
        → 답을 모를 경우 모른다고 명시하도록 합니다.
      - “Don't try to make up an answer.”  
        → 허구의 답변을 만들지 말라는 경고입니다.
      - “------” 이후 `{context}` 플레이스홀더가 있어, 추출된 문맥이 이 자리에 삽입됩니다.
    
- **라인 78:**  
  ```python
      ("human", "{question}")
  ```  
  - **역할:**  
    - `"human"` 메시지로, 사용자 질문을 `{question}` 플레이스홀더에 넣습니다.

- **라인 79:**  
  ```python
  ])
  ```  
  - **역할:**  
    - 프롬프트 템플릿 구성을 마칩니다.
    - `final_prompt`는 이후 최종 답변 생성을 위한 지침과 사용자 질문을 포함하는 템플릿이 됩니다.

---

## 6. 최종 체인 구성 및 실행 (라인 81 ~ 83)

- **라인 81:**  
  ```python
  chain = {"context": map_chain, "question": RunnablePassthrough()} | final_prompt | llm
  ```  
  - **역할:**  
    - 최종적인 체인을 구성합니다.
  - **세부 설명:**  
    - **입력 구성:**  
      - 딕셔너리 `{ "context": map_chain, "question": RunnablePassthrough() }`  
        - `"context": map_chain`  
          - 이전 단계에서 구성한 `map_chain`이 최종 프롬프트의 `{context}`에 들어갈 데이터를 생성합니다.  
          - 이 과정에서 retriever를 통해 관련 문서를 검색하고, 각 문서에서 질문과 관련된 텍스트를 추출해 통합합니다.
        - `"question": RunnablePassthrough()`  
          - 사용자의 질문을 그대로 다음 단계로 전달합니다.
    - **파이프 연산자(`|`)의 연결 순서:**  
      1. **첫번째 연결:**  
         - 입력 딕셔너리를 `final_prompt`에 전달하여, 템플릿에 `{context}`와 `{question}`을 채워넣습니다.
      2. **두번째 연결:**  
         - 완성된 프롬프트를 LLM(`llm`)에 전달하여 최종 답변을 생성하도록 합니다.
    - **최종 출력:**  
      - LLM이 생성한 최종 답변(텍스트)이 체인의 결과로 반환됩니다.

- **라인 83:**  
  ```python
  chain.invoke("What is the name of Oldwyn’s mechanical doll?")
  ```  
  - **역할:**  
    - 최종 체인을 실행(호출)합니다.
  - **세부 처리 과정:**  
    1. **입력:**  
       - 문자열 `"What is the name of Oldwyn’s mechanical doll?"`가 전달됩니다.
    2. **처리:**  
       - 이 질문은 `RunnablePassthrough()`를 통해 그대로 전달되며, retriever를 통해 관련 문서들이 검색됩니다.
       - `map_chain`이 각 문서에서 관련 텍스트를 추출해 하나의 문맥으로 통합합니다.
       - 이 통합된 문맥과 원래 질문이 `final_prompt`에 채워져 최종 프롬프트가 생성되고, LLM이 이를 바탕으로 답변을 만듭니다.
    3. **출력:**  
       - LLM이 최종적으로 생성한 답변이 반환됩니다.
  - **의미:**  
    - 전체 체인 과정을 통해, 주어진 질문에 대해 관련 문서에서 필요한 정보를 추출하고 최종적으로 정확한 답변을 생성하게 됩니다.

---

이와 같이, 37번째 줄부터 84번째 줄까지의 코드는 다음과 같은 큰 흐름을 따릅니다:

1. **문서의 관련 텍스트 추출:**  
   - 각 문서 조각에 대해 LLM이 질문과 관련된 부분을 찾아 반환하도록 함.
2. **추출된 텍스트의 통합:**  
   - 여러 문서에서 나온 결과를 하나의 큰 문맥으로 합침.
3. **최종 답변 생성:**  
   - 통합된 문맥과 질문을 바탕으로 LLM이 최종 답변을 생성함.

이로써, 질문 `"What is the name of Oldwyn’s mechanical doll?"`에 대해 체계적으로 관련 정보를 검색하고, 추출한 후 최종 답변을 만들어내는 전체 파이프라인이 완성됩니다.